In [1]:
import nmrglue as ng
import numpy as np
from cops_analysis import cops_analyze
import pandas as pd
import matplotlib.pyplot as plt
import os
import pluqin

In [2]:
tb = pd.read_excel('CB_fit_results_nopyruvate.xlsx')
CACB = tb[['CA','CB']].to_numpy()
tb_predict = pd.read_csv('cs_stat_aa_filt.csv')
CACB=CACB+2
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

tb_predict = tb_predict.drop(tb_predict[tb_predict['name']=='G'].index)
means_CA = tb_predict[['mean']][tb_predict['atom name']=='CA'].to_numpy()
stdevs_CA = tb_predict[['stdev']][tb_predict['atom name']=='CA'].to_numpy()
means_CB = tb_predict[['mean']][tb_predict['atom name']=='CB'].to_numpy()
stdevs_CB = tb_predict[['stdev']][tb_predict['atom name']=='CB'].to_numpy()
labels = tb_predict[['name']][tb_predict['atom name']=='CA'].to_numpy()
probs = np.multiply(gaussian(CACB[:,0], means_CA, stdevs_CA),gaussian(CACB[:,1],means_CB, stdevs_CB))
probs = np.divide(probs.T,np.sum(probs.T, axis=1).reshape([-1,1]))
probs = np.round(probs, 2)
probs = [np.dstack([labels.T, probs[i]]).reshape([-1,2])[np.argsort(-probs[i])] for i in range(len(probs))]
tb['gaussian predicts'] = probs

In [104]:
tb

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Assignment,CA,HN,CB,1/covariance,CB_scott,pluq_result,gaussian predicts
0,0,0,0,T1Ca-HN,52.978,8.284,27.330293,6.878374,27.024,"[['Q', 36.3], ['H', 29.6], ['E', 17.3], ['R', ...","[[Q, 0.19], [H, 0.19], [R, 0.14], [C, 0.14], [..."
1,1,1,1,T1Ca-Y2HN,52.978,9.048,27.131355,4.908550,27.024,"[['Q', 35.2], ['H', 34.5], ['E', 15.0], ['R', ...","[[Q, 0.2], [H, 0.19], [C, 0.15], [R, 0.14], [W..."
2,2,2,2,Y2Ca-HN,54.261,9.048,39.950039,13.657712,39.978,"[['L', 55.1], ['D', 22.9], ['Y', 21.3], ['C', ...","[[L, 0.34], [D, 0.21], [F, 0.17], [C, 0.14], [..."
3,3,3,3,Y2Ca-K3HN,54.261,9.048,39.950039,13.657712,39.978,"[['L', 55.1], ['D', 22.9], ['Y', 21.3], ['C', ...","[[L, 0.34], [D, 0.21], [F, 0.17], [C, 0.14], [..."
4,4,4,4,K3Ca-HN,52.176,9.047,32.394571,10.822657,32.390,"[['K', 33.5], ['R', 23.6], ['E', 23.4], ['M', ...","[[C, 0.33], [M, 0.33], [K, 0.18], [H, 0.05], [..."
...,...,...,...,...,...,...,...,...,...,...,...
104,104,104,104,V53Ca-HN,55.099,8.179,28.863622,4.908015,28.760,"[['E', 37.6], ['R', 24.8], ['H', 10.8], ['K', ...","[[R, 0.15], [H, 0.15], [W, 0.14], [C, 0.14], [..."
105,105,105,105,V53Ca-T54HN,55.089,8.303,28.864995,5.835914,28.760,"[['E', 37.7], ['R', 24.8], ['H', 10.8], ['K', ...","[[R, 0.15], [H, 0.15], [W, 0.14], [C, 0.14], [..."
106,106,106,106,T54Ca-HN,57.846,8.304,45.178455,5.172885,67.260,"['T', 1]","[[C, 0.94], [L, 0.03], [F, 0.01], [Y, 0.01], [..."
107,107,107,107,T54Ca-E55HN,57.846,8.088,44.985719,5.854357,67.260,"['T', 1]","[[C, 0.93], [L, 0.04], [F, 0.02], [Y, 0.01], [..."


In [99]:
tb.to_excel('CB_fit_results_nopyruvate.xlsx')

In [3]:
pluq_result=[]
for i in range(len(tb)):
    if CACB[i][1]==0:
        pluq_result.append(['G',1])
    elif CACB[i][0]>54 and CACB[i][1]>44:
        pluq_result.append(['T',1])
    else:
        pluq_out = pluqin.main([CACB[i]], 'cc')
        if pluq_out!=None:
            pluq_result.append([[pluq_out[i][0], pluq_out[i][2]] for i in range(len(pluq_out))])
        else:
            pluq_result.append(['X',1])
    
tb['pluq_result'] = pluq_result

In [31]:
j = tb.loc[1].pluq_result

In [32]:
pyruvate_posterior(j, 0.35, df)

[['Q', 35.2], ['H', 34.5], ['E', 15.0], ['R', 8.3], ['W', 3.7], ['M', 2.3], ['C', 1.0]]
<class 'list'>
(7, 2)
['Q' 'H' 'E' 'R' 'W' 'M' 'C']
        Residue  mean  stdev  conditional_prob
Residue                                       
Q             Q    35      5      1.784124e-01
H             H    10      5      6.648812e-07
E             E    35      5      1.784124e-01
R             R    35      5      1.784124e-01
W             W    20      5      1.981983e-03
M             M    40      5      1.082126e-01
C             C    15      5      5.985070e-05


array([['Q', '0.59'],
       ['E', '0.25'],
       ['R', '0.14'],
       ['M', '0.02'],
       ['H', '0.0'],
       ['W', '0.0'],
       ['C', '0.0']], dtype='<U32')

In [149]:
tb.to_excel('CB_fit_results_nopyruvate.xlsx')

In [26]:
df = pd.read_excel('pyruvate_likelihood.xlsx')
df.index = df['Residue']

def pyruvate_posterior(input_prob, pyruvate_frac, df):
    print(input_prob)
    print(type(input_prob))
    
    input_prob = np.array(input_prob)
    print(input_prob.shape)
    df['conditional_prob']=gaussian(100*pyruvate_frac, df['mean'].to_numpy(), df['stdev'].to_numpy())
    print(input_prob[:,0])
    print(df.loc[list(input_prob[:,0])])
    posterior = input_prob[:,1].astype('float64')*df.loc[input_prob[:,0]]['conditional_prob']
    posterior = posterior/np.sum(posterior)
    input_prob[:,1] = posterior.round(2)
    posterior = input_prob
    posterior = posterior[np.argsort(-posterior[:,1].astype('float64'))]
    return posterior

def gaussian(x, mu, sig):
    return 1/np.sqrt(2*np.pi*sig)*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))